<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="data/D4Sci_logo_ball.png" alt="Data For Science, Inc" align="left" border="0"> </div>
    <div style="float: left; margin-left: 10px;"> <h1>Natural Language Processing</h1>
<h1>NLP with LLMs</h1>
        <p>Bruno Gonçalves<br/>
        <a href="http://www.data4sci.com/">www.data4sci.com</a><br/>
            @bgoncalves, @data4sci</p></div>
</div>

In [1]:
from collections import Counter, defaultdict
from pprint import pprint
import random

import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt 

import nltk
from nltk.corpus import reuters
from nltk import bigrams, trigrams

import torch
import transformers
from transformers import pipeline
from transformers import set_seed
set_seed(42) # Set the seed to get reproducible results

import tqdm as tq
from tqdm.notebook import tqdm
tqdm.pandas()

import watermark

%load_ext watermark
%matplotlib inline

We start by printing out the versions of the libraries we're using for future reference

In [2]:
%watermark -n -v -m -g -iv

Python implementation: CPython
Python version       : 3.11.7
IPython version      : 8.12.3

Compiler    : Clang 14.0.6 
OS          : Darwin
Release     : 24.3.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit

Git hash: 78c5d339e4008417c5104695fabcddc9da4faed0

torch       : 2.3.0
tqdm        : 4.66.4
watermark   : 2.4.3
json        : 2.0.9
matplotlib  : 3.8.0
nltk        : 3.8.1
pandas      : 2.2.3
transformers: 4.41.1
numpy       : 1.26.4



Load default figure style

In [3]:
plt.style.use('d4sci.mplstyle')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# "Small" Language Model

In [4]:
model = defaultdict(lambda: defaultdict(lambda: 0))

We start by counting number of trigram co-occurrences

In [5]:
for sentence in tqdm(reuters.sents(), total=54_716):
    for w1, w2, w3 in trigrams(sentence, pad_right=True, pad_left=True):
        bigram = (w1, w2)
        model[bigram][w3] += 1

  0%|          | 0/54716 [00:00<?, ?it/s]

And normalizing the probabilities for each bigram. 

In [6]:
for bigram in model:
    total_count = float(sum(model[bigram].values()))

    for w3 in model[bigram]:
        model[bigram][w3] /= total_count

Our language model is just a weighted mapping between each bigram and the possible next words.

In [7]:
model[("the", "United")]

defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
            {'States': 0.880672268907563,
             'Kingdom': 0.011764705882352941,
             'Arab': 0.052100840336134456,
             'Permanent': 0.0016806722689075631,
             'Steelworkers': 0.0033613445378151263,
             'Nations': 0.025210084033613446,
             'Coconut': 0.0067226890756302525,
             'State': 0.0033613445378151263,
             'Democratic': 0.0016806722689075631,
             'Food': 0.008403361344537815,
             'Automobile': 0.0016806722689075631,
             'acquisition': 0.0016806722689075631,
             'Rubber': 0.0016806722689075631})

In [8]:
model[("United", "Kingdom")]

defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
            {',': 0.21428571428571427,
             'and': 0.21428571428571427,
             'blender': 0.07142857142857142,
             ')': 0.14285714285714285,
             'company': 0.07142857142857142,
             'operations': 0.07142857142857142,
             'assets': 0.07142857142857142,
             'Ltd': 0.07142857142857142,
             '.': 0.07142857142857142})

This is all we need to generate new text staring from a bigram prompt. We must simply perform a random walk on this weighted graph starting from an initial prompt:

In [9]:
def generate_sentence_from_prompt(prompt, zero_temperature=False):
    text = [*prompt]

    # Dont impose any fixed sentence length
    while True:
        # the current not we're in is just the one that accounts
        # for the last two words in the text
        bigram = tuple(text[-2:])

        # We extract the list of possible next words and their probabilities
        words = []
        probs = []

        for word, prob in model[bigram].items():
            words.append(word)
            probs.append(prob)

        # Choose one word proportionally to each probability
        selection = np.random.multinomial(1, probs)
        
        # Check which one was chosen
        if zero_temperature:
            pos = np.argmax(probs) # Temperature = 0
        else:
            pos = np.argmax(selection) # Temperature = 1

        word = words[pos]

        # Append the new word to our runnning text
        text.append(word)

        # Wtop when we hit two None tokens in a row, that represnet the end of a sentence
        if text[-2:] == [None, None]:
            break
        
        # Make sure we don't run forever
        if zero_temperature and len(text) > 100:
            break
                
    return " ".join([t for t in text if t])

In [10]:
generate_sentence_from_prompt(('United', 'States'))

"United States uses only six pct of Belkin ' s annual stockholders ' equity was 2 . 7 mln CPC shares , or a combination of the fraction of a weaker currency states , where rising production has dropped its insistence on repaying the loan repayment rate for February , down from last year after it completes a proposed tax would depress the dollar will stimulate export demand , which reaffirmed support for the remainder ."

In [11]:
generate_sentence_from_prompt(('today', 'the'))

'today the Italian bank was planning a branch in the Business Administration Court in Luxembourg released here , many irritants remain .'

In [12]:
generate_sentence_from_prompt(('financial', 'markets'))

'financial markets as an intermediate model between its lightest and heaviest crudes .'

# Named Entity Recognition

In [13]:
email = """Dear Amazon, \

last week I ordered an Optimus Prime action figure \
from your online store in Germany. Unfortunately, when I opened the package, \
I discovered to my horror that I had been sent an action figure of Megatron \
instead! As a lifelong enemy of the Decepticons, I hope you can understand my \
dilemma. To resolve the issue, I demand an exchange of Megatron for the \
Optimus Prime figure I ordered. Enclosed are copies of my records concerning \
this purchase. I expect to hear from you soon. 

Sincerely, 

Bumblebee."""

In [14]:
ner_tagger = pipeline("ner", aggregation_strategy="simple")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/opt/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTra

In [15]:
outputs = ner_tagger(email)

In [16]:
outputs

[{'entity_group': 'ORG',
  'score': 0.8790102,
  'word': 'Amazon',
  'start': 5,
  'end': 11},
 {'entity_group': 'MISC',
  'score': 0.9908588,
  'word': 'Optimus Prime',
  'start': 37,
  'end': 50},
 {'entity_group': 'LOC',
  'score': 0.9997547,
  'word': 'Germany',
  'start': 91,
  'end': 98},
 {'entity_group': 'MISC',
  'score': 0.5565716,
  'word': 'Mega',
  'start': 209,
  'end': 213},
 {'entity_group': 'PER',
  'score': 0.59025526,
  'word': '##tron',
  'start': 213,
  'end': 217},
 {'entity_group': 'ORG',
  'score': 0.66969275,
  'word': 'Decept',
  'start': 254,
  'end': 260},
 {'entity_group': 'MISC',
  'score': 0.4983484,
  'word': '##icons',
  'start': 260,
  'end': 265},
 {'entity_group': 'MISC',
  'score': 0.7753625,
  'word': 'Megatron',
  'start': 351,
  'end': 359},
 {'entity_group': 'MISC',
  'score': 0.98785394,
  'word': 'Optimus Prime',
  'start': 368,
  'end': 381},
 {'entity_group': 'PER',
  'score': 0.8120968,
  'word': 'Bumblebee',
  'start': 507,
  'end': 516}]

In [17]:
pd.DataFrame(outputs)    

,entity_group,score,word,start,end
0,ORG,0.879010,Amazon,5,11
1,MISC,0.990859,Optimus Prime,37,50
2,LOC,0.999755,Germany,91,98
3,MISC,0.556572,Mega,209,213
4,PER,0.590255,##tron,213,217
5,ORG,0.669693,Decept,254,260
6,MISC,0.498348,##icons,260,265
7,MISC,0.775362,Megatron,351,359
8,MISC,0.987854,Optimus Prime,368,381
9,PER,0.812097,Bumblebee,507,516


# PoS Tagging

Load the pipeline

In [18]:
pos_tagger = pipeline("token-classification", model="vblagoje/bert-english-uncased-finetuned-pos")

Some weights of the model checkpoint at vblagoje/bert-english-uncased-finetuned-pos were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
text = "The quick brown fox jumps over the lazy dog."

Extract the part of speech tags

In [20]:
pos_tags = pos_tagger(text)
pd.DataFrame(pos_tags)

,entity,score,index,word,start,end
0,DET,0.999445,1,the,0,3
1,ADJ,0.997063,2,quick,4,9
2,ADJ,0.942299,3,brown,10,15
3,NOUN,0.997004,4,fox,16,19
4,VERB,0.999446,5,jumps,20,25
5,ADP,0.999325,6,over,26,30
6,DET,0.999527,7,the,31,34
7,ADJ,0.997863,8,lazy,35,39
8,NOUN,0.998858,9,dog,40,43
9,PUNCT,0.999650,10,.,43,44


# Summarization

In [21]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
/opt/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


The first 4 paragraphs of https://en.wikipedia.org/wiki/Transformers

In [22]:
wiki_text = """
Transformers is a media franchise produced by American toy company Hasbro and Japanese toy company Takara Tomy. It primarily follows the heroic Autobots and the villainous Decepticons, two alien robot factions at war that can transform into other forms, such as vehicles and animals. The franchise encompasses toys, animation, comic books, video games and films. As of 2011, it generated more than ¥2 trillion ($25 billion) in revenue,[1] making it one of the highest-grossing media franchises of all time.

The franchise began in 1984 with the Transformers toy line, comprising transforming mecha toys from Takara's Diaclone and Micro Change toylines rebranded for Western markets.[2] The term "Generation 1" (G1) covers both the animated television series The Transformers and the comic book series of the same name, which are further divided into Japanese, British and Canadian spin-offs. Sequels followed, such as the Generation 2 comic book and Beast Wars TV series, which became its own mini-universe. Generation 1 characters have been rebooted multiple times in the 21st century in comics from Dreamwave Productions (starting 2001), IDW Publishing (starting in 2005 and again in 2019), and Skybound Entertainment (beginning in 2023). There have been other incarnations of the story based on different toy lines during and after the 20th century. The first was the Robots in Disguise series, followed by three shows (Armada, Energon, and Cybertron) that constitute a single universe called the "Unicron Trilogy".

A live-action film series started in 2007, again distinct from previous incarnations, while the Transformers: Animated series merged concepts from the G1 continuity, the 2007 live-action film and the "Unicron Trilogy". For most of the 2010s, in an attempt to mitigate the wave of reboots, the "Aligned Continuity" was established. In 2018, Transformers: Cyberverse debuted, once again, distinct from the previous incarnations.

Although a separate and competing franchise started in 1983, Tonka's GoBots became the intellectual property of Hasbro after their buyout of Tonka in 1991. Subsequently, the universe depicted in the animated series Challenge of the GoBots and follow-up film GoBots: Battle of the Rock Lords was retroactively established as an alternate universe within the Transformers multiverse.[3] 
"""

To generate the summary we just have to call the pipeline

In [23]:
summary = summarizer(wiki_text)

print(summary[0]['summary_text'])

 The Transformers is a media franchise produced by Hasbro and Japanese toy company Takara Tomy . It primarily follows the heroic Autobots and the villainous Decepticons, two alien robot factions at war that can transform into other forms, such as vehicles and animals . As of 2011, it generated more than ¥2 trillion ($25 billion) in revenue .


We can also specify a minimum length

In [24]:
summary = summarizer(wiki_text, min_length=100)

print(summary[0]['summary_text'])

 Transformers is a media franchise produced by Hasbro and Japanese toy company Takara Tomy . It primarily follows the heroic Autobots and the villainous Decepticons, two alien robot factions at war that can transform into other forms, such as vehicles and animals . As of 2011, it generated more than ¥2 trillion ($25 billion) in revenue, making it one of the highest-grossing media franchises of all time . The term "Generation 1" (G1) covers both the animated television series The Transformers and the comic book series of the same name .


# Question Answering 

In [25]:
reader = pipeline("question-answering")

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [26]:
question = "What does the customer want?"

In [27]:
outputs = reader(question=question, context=email)
pd.DataFrame([outputs])    

,score,start,end,answer
0,0.631292,336,359,an exchange of Megatron


# Translation

In [28]:
translator = pipeline("translation_en_to_it", 
                      model="Helsinki-NLP/opus-mt-en-it")

In [29]:
outputs = translator(email, clean_up_tokenization_spaces=True, min_length=100, max_length=1000)
print(outputs[0]['translation_text'])

Cara Amazon, la scorsa settimana ho ordinato una figura d'azione Optimus Prime dal tuo negozio online in Germania. Purtroppo, quando ho aperto il pacchetto, ho scoperto al mio orrore che ero stato inviato una figura d'azione di Megatron invece! Come un nemico per tutta la vita dei Decepticon, spero che si può capire il mio dilemma. Per risolvere il problema, chiedo uno scambio di Megatron per la figura di Optimus Prime ho ordinato. In allegato sono copie dei miei record riguardanti questo acquisto. Mi aspetto di sentire da voi presto. Cordialmente, Bumblebee.


For comparison, let us look at the results of google translate:

```
Caro Amazon, la settimana scorsa ho ordinato un action figure di Optimus Prime dal tuo negozio online in Germania. Sfortunatamente, quando ho aperto il pacco, ho scoperto con orrore che mi era stata invece inviata una action figure di Megatron! Essendo un nemico da sempre dei Decepticon, spero che tu possa capire il mio dilemma. Per risolvere il problema, chiedo uno scambio di Megatron con la figura di Optimus Prime che ho ordinato. In allegato sono presenti copie dei miei documenti relativi a questo acquisto. Mi aspetto di sentirti presto. Cordiali saluti, Bombo.
```

Google translate is less context aware in the translation going so far as translating the name of the email sender (Bumblebee -> Bombo). On the other hand, the Hugging Face model is more formal ("sentire da voi" -> "sentirti")

# Sentiment Analysis

In [30]:
sentiment_pipeline = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/opt/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Let us use a comple of faily obvious instances

In [31]:
instances = ["I love you", "I hate you"]

The model does a pretty good job of figuring out which one is positive and which one is negative

In [32]:
sentiment_pipeline(instances)

[{'label': 'POSITIVE', 'score': 0.9998656511306763},
 {'label': 'NEGATIVE', 'score': 0.9991129040718079}]

# Application

Load a few thousand tweets about Apple

In [33]:
data = pd.read_csv('data/Apple-Twitter-Sentiment-DFE.csv', usecols=['text'])

In [34]:
data

,text
0,#AAPL:The 10 best Steve Jobs emails ever...htt...
1,RT @JPDesloges: Why AAPL Stock Had a Mini-Flas...
2,My cat only chews @apple cords. Such an #Apple...
3,I agree with @jimcramer that the #IndividualIn...
4,Nobody expects the Spanish Inquisition #AAPL
...,...
3881,(Via FC) Apple Is Warming Up To Social Media -...
3882,RT @MMLXIV: there is no avocado emoji may I as...
3883,@marcbulandr I could not agree more. Between @...
3884,My iPhone 5's photos are no longer downloading...


Compute the sentiment score for each tweet

In [35]:
sent = pd.DataFrame(data['text'].progress_apply(lambda x: pd.Series(sentiment_pipeline(x)[0])))

In [36]:
sent.rename(columns={'score': 'sentiment_confidence', 'label':'sentiment'}, inplace=True)

In [37]:
sent

,sentiment,sentiment_confidence
0,POSITIVE,0.999432
1,NEGATIVE,0.999122
2,NEGATIVE,0.996177
3,POSITIVE,0.995648
4,NEGATIVE,0.932676
...,...,...
3881,NEGATIVE,0.992046
3882,NEGATIVE,0.999158
3883,NEGATIVE,0.935773
3884,NEGATIVE,0.998303


We can also use NER to identify when a person is mentioned in the tweet

In [38]:
data['text'].iloc[0]

'#AAPL:The 10 best Steve Jobs emails ever...http://t.co/82G1kL94tx'

In [39]:
ner_tagger(data['text'].iloc[0])

[{'entity_group': 'PER',
  'score': 0.73089933,
  'word': 'Steve Jobs',
  'start': 18,
  'end': 28}]

Identify all people mentioned

In [40]:
def find_people(x):
    output = ner_tagger(x)
    
    for tag in output:
        if tag['entity_group'] == 'PER':
            out = {'confidence':tag["score"], 'person': tag['word']}
            return pd.Series(out)
    
    return pd.Series({"confidence": None, "person": None})

In [41]:
people = pd.DataFrame(data['text'].progress_apply(find_people))

  0%|          | 0/3886 [00:00<?, ?it/s]

Combine all the results into a single DataFrame

In [42]:
data = pd.concat([data, sent, people], axis=1)

In [43]:
data

,text,sentiment,sentiment_confidence,confidence,person
0,#AAPL:The 10 best Steve Jobs emails ever...htt...,POSITIVE,0.999432,0.730899,Steve Jobs
1,RT @JPDesloges: Why AAPL Stock Had a Mini-Flas...,NEGATIVE,0.999122,NaN,None
2,My cat only chews @apple cords. Such an #Apple...,NEGATIVE,0.996177,NaN,None
3,I agree with @jimcramer that the #IndividualIn...,POSITIVE,0.995648,NaN,None
4,Nobody expects the Spanish Inquisition #AAPL,NEGATIVE,0.932676,NaN,None
...,...,...,...,...,...
3881,(Via FC) Apple Is Warming Up To Social Media -...,NEGATIVE,0.992046,NaN,None
3882,RT @MMLXIV: there is no avocado emoji may I as...,NEGATIVE,0.999158,NaN,None
3883,@marcbulandr I could not agree more. Between @...,NEGATIVE,0.935773,NaN,None
3884,My iPhone 5's photos are no longer downloading...,NEGATIVE,0.998303,NaN,None


Subset the data to only the tweets meantioning people

In [44]:
people = data[data.person.isna() == False].copy()

In [45]:
people

,text,sentiment,sentiment_confidence,confidence,person
0,#AAPL:The 10 best Steve Jobs emails ever...htt...,POSITIVE,0.999432,0.730899,Steve Jobs
17,@Apple John Cantlie has been a prisoner of ISI...,POSITIVE,0.930674,0.984458,John Cantlie
80,I'm hoping @apple won't automatically make us ...,NEGATIVE,0.997191,0.889901,Bruce Springsteen
81,@thehill @Apple What a joke! Justice Dept shou...,NEGATIVE,0.992056,0.941649,Killer Wilson
85,#AAPL:10 Steve Jobs emails you need to read......,NEGATIVE,0.994819,0.495679,Job
...,...,...,...,...,...
3777,Jeff Daniels in Talks to Play Former Apple CEO...,NEGATIVE,0.593705,0.999621,Jeff Daniels
3798,"Dear Santa, All I want for Christmas is for @A...",NEGATIVE,0.979491,0.839571,Santa
3812,@Apple co-founder Steve Wozniak talks about St...,POSITIVE,0.938843,0.941354,Steve Wozniak
3818,RT @CNET: @Apple pioneer Bill Fernandez on @Go...,NEGATIVE,0.922325,0.999579,Bill Fernandez


Convert the text labels to a numerical score

In [46]:
people['sentiment'] = people.apply(lambda x: 1 if x.sentiment == 'POSITIVE' else -1, axis=1)

Compute the average score

In [47]:
stats = people[['person', 'sentiment']].groupby('person').mean()

In [48]:
counts = people[['person', 'sentiment']].groupby('person').count()
counts.rename(columns={'sentiment':'count'}, inplace=True)

In [49]:
stats = stats.join(counts)

In [50]:
stats[stats['count']>=5].sort_values('sentiment', ascending=False)

,sentiment,count
person,,
Steve Wozniak,-0.200000,10
Steve,-0.333333,6
Eddy Cue,-0.500000,8
Steve Job,-0.647059,17
Mark Zuckerberg,-0.714286,7
Steve Jobs,-0.777778,27
Tim Cook,-0.793103,29
Job,-0.800000,10


<center>
     <img src="data/D4Sci_logo_full.png" alt="Data For Science, Inc" align="center" border="0" width=300px> 
</center>